<a href="https://colab.research.google.com/github/MHC-FA24-CS341CV/beyond-the-pixels-emerging-computer-vision-research-topics-fa24/blob/main/GLIDE_Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Inpainting Using GLIDE
A more user friendly notebook derived from the original in the **glide-text2im** GitHub repo, that shows how to use filtered GLIDE to fill in a masked region of an image, conditioned on a text prompt.  
Here's the [link](https://arxiv.org/abs/2112.10741) to the paper.  
The code in this notebook supports execution on CPU or GPU. Anyway, A GPU runtime is suggested to keep the sampling execution within a minute.

Clone the official GitHub repository.

In [ ]:
!git clone https://github.com/openai/glide-text2im.git
%cd glide-text2im

Cloning into 'glide-text2im'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 76 (delta 26), reused 22 (delta 22), pack-reused 32 (from 1)
Receiving objects: 100% (76/76), 2.01 MiB | 7.77 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/glide-text2im


## Settings

Add the *glide_text2im* package to the Python *sys.path*.

In [ ]:
%set_env PYTHONPATH=/content/packnet-sfm/packnet_sfm:/env/python

env: PYTHONPATH=/content/packnet-sfm/packnet_sfm:/env/python


Import the necessary packages.

In [ ]:
from typing import Tuple

from IPython.display import display
from PIL import Image
import numpy as np
import torch as th
import torch.nn.functional as F

from glide_text2im.download import load_checkpoint
from glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
    model_and_diffusion_defaults_upsampler
)

Check if a GPU is available. If not, switch execution to CPU.

In [ ]:
has_cuda = th.cuda.is_available()
device = th.device('cpu' if not has_cuda else 'cuda')

Create the base model.

In [5]:
options = model_and_diffusion_defaults()
options['inpaint'] = True
options['use_fp16'] = has_cuda
options['timestep_respacing'] = '100' # use 100 diffusion steps for fast sampling
model, diffusion = create_model_and_diffusion(**options)
model.eval()
if has_cuda:
    model.convert_to_fp16()
model.to(device)
model.load_state_dict(load_checkpoint('base-inpaint', device))
print('total base parameters', sum(x.numel() for x in model.parameters()))

  0%|          | 0.00/1.54G [00:00<?, ?iB/s]

/content/glide-text2im/glide_text2im/download.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return th.load(path, map_location=device)


total base parameters 385037638


Create the upsampler model.

In [ ]:
options_up = model_and_diffusion_defaults_upsampler()
options_up['inpaint'] = True
options_up['use_fp16'] = has_cuda
options_up['timestep_respacing'] = 'fast27' # use 27 diffusion steps for very fast sampling
model_up, diffusion_up = create_model_and_diffusion(**options_up)
model_up.eval()
if has_cuda:
    model_up.convert_to_fp16()
model_up.to(device)
model_up.load_state_dict(load_checkpoint('upsample-inpaint', device))
print('total upsampler parameters', sum(x.numel() for x in model_up.parameters()))

  0%|          | 0.00/1.59G [00:00<?, ?iB/s]

Define a function to display a batch of images inline.

In [ ]:
def show_images(batch: th.Tensor):
    """ Display a batch of images inline. """
    scaled = ((batch + 1)*127.5).round().clamp(0,255).to(th.uint8).cpu()
    reshaped = scaled.permute(2, 0, 3, 1).reshape([batch.shape[2], -1, 3])
    display(Image.fromarray(reshaped.numpy()))

Define a function to convert an image into a multi-dimentional array.

In [ ]:
def read_image(path: str, size: int = 256) -> Tuple[th.Tensor, th.Tensor]:
    pil_img = Image.open(path).convert('RGB')
    pil_img = pil_img.resize((size, size), resample=Image.BICUBIC)
    img = np.array(pil_img)
    return th.from_numpy(img)[None].permute(0, 3, 1, 2).float() / 127.5 - 1

Upload the source image to impaint.

In [ ]:
from google.colab import files

uploaded = files.upload()


Implement a form to set the values for the sampling parameters.

In [ ]:
#@title Sampling Parameters

prompt = "" #@param {type:"string"}
batch_size = 1 #@param {type:"slider", min:1, max:8, step:1}
guidance_scale = 3 #@param {type:"slider", min:1.0, max:10.0, step:0.1}
upsample_temp = 0.997 #@param {type:"slider", min:0, max:1, step:0.1}

Create the mask and display the source image with the mask on top of it.

In [ ]:
for key in uploaded.keys():
  uploaded_image = key

In [ ]:
# Source image we are inpainting
source_image_256 = read_image(uploaded_image, size=256)
source_image_64 = read_image(uploaded_image, size=64)

# The mask should always be a boolean 64x64 mask, and then we
# can upsample it for the second stage.
source_mask_64 = th.ones_like(source_image_64)[:, :1]
source_mask_64[:, :, 20:] = 0
source_mask_256 = F.interpolate(source_mask_64, (256, 256), mode='nearest')

# Visualize the image we are inpainting
show_images(source_image_256 * source_mask_256)

## Sampling from Base Model

Create the text tokens to feed to the model.

In [ ]:
tokens = model.tokenizer.encode(prompt)
tokens, mask = model.tokenizer.padded_tokens_and_mask(
    tokens, options['text_ctx']
)

Create the empty classifier-free guidance tokens.

In [ ]:
full_batch_size = batch_size * 2
uncond_tokens, uncond_mask = model.tokenizer.padded_tokens_and_mask(
    [], options['text_ctx']
)

Pack the tokens together into model *kwargs*.

In [ ]:
model_kwargs = dict(
    tokens=th.tensor(
        [tokens] * batch_size + [uncond_tokens] * batch_size, device=device
    ),
    mask=th.tensor(
        [mask] * batch_size + [uncond_mask] * batch_size,
        dtype=th.bool,
        device=device,
    ),

    # Masked inpainting image
    inpaint_image=(source_image_64 * source_mask_64).repeat(full_batch_size, 1, 1, 1).to(device),
    inpaint_mask=source_mask_64.repeat(full_batch_size, 1, 1, 1).to(device),
)

Implement a classifier-free guidance sampling function.

In [ ]:
def model_fn(x_t, ts, **kwargs):
    half = x_t[: len(x_t) // 2]
    combined = th.cat([half, half], dim=0)
    model_out = model(combined, ts, **kwargs)
    eps, rest = model_out[:, :3], model_out[:, 3:]
    cond_eps, uncond_eps = th.split(eps, len(eps) // 2, dim=0)
    half_eps = uncond_eps + guidance_scale * (cond_eps - uncond_eps)
    eps = th.cat([half_eps, half_eps], dim=0)
    return th.cat([eps, rest], dim=1)

def denoised_fn(x_start):
    # Force the model to have the exact right x_start predictions
    # for the part of the image which is known.
    return (
        x_start * (1 - model_kwargs['inpaint_mask'])
        + model_kwargs['inpaint_image'] * model_kwargs['inpaint_mask']
    )

Sample from the base model.

In [ ]:
model.del_cache()
samples = diffusion.p_sample_loop(
    model_fn,
    (full_batch_size, 3, options["image_size"], options["image_size"]),
    device=device,
    clip_denoised=True,
    progress=True,
    model_kwargs=model_kwargs,
    cond_fn=None,
    denoised_fn=denoised_fn,
)[:batch_size]
model.del_cache()

Display the result.

In [ ]:
show_images(samples)

## Upsampling the 64x64 Samples

Create the text tokens to feed to the model.

In [ ]:
tokens = model_up.tokenizer.encode(prompt)
tokens, mask = model_up.tokenizer.padded_tokens_and_mask(
    tokens, options_up['text_ctx']
)

Create the model conditioning dict.

In [ ]:
model_kwargs = dict(
    # Low-res image to upsample.
    low_res=((samples+1)*127.5).round()/127.5 - 1,

    # Text tokens
    tokens=th.tensor(
        [tokens] * batch_size, device=device
    ),
    mask=th.tensor(
        [mask] * batch_size,
        dtype=th.bool,
        device=device,
    ),

    # Masked inpainting image.
    inpaint_image=(source_image_256 * source_mask_256).repeat(batch_size, 1, 1, 1).to(device),
    inpaint_mask=source_mask_256.repeat(batch_size, 1, 1, 1).to(device),
)

Define a function to force the model to have the exact right *x_start* predictions for the part of the image which is known.

In [ ]:
def denoised_fn(x_start):
    return (
        x_start * (1 - model_kwargs['inpaint_mask'])
        + model_kwargs['inpaint_image'] * model_kwargs['inpaint_mask']
    )

Sampling.

In [ ]:
model_up.del_cache()
up_shape = (batch_size, 3, options_up["image_size"], options_up["image_size"])
up_samples = diffusion_up.p_sample_loop(
    model_up,
    up_shape,
    noise=th.randn(up_shape, device=device) * upsample_temp,
    device=device,
    clip_denoised=True,
    progress=True,
    model_kwargs=model_kwargs,
    cond_fn=None,
    denoised_fn=denoised_fn,
)[:batch_size]
model_up.del_cache()

Display the result.

In [ ]:
show_images(up_samples)

In [ ]:
import cv2
import numpy as np
import torch

def save_image(image_array, filename="img.png"):
    # If image_array is a PyTorch tensor, convert it to a NumPy array
    if isinstance(image_array, torch.Tensor):
        # Remove the batch dimension (from [1, 3, 256, 256] to [3, 256, 256])
        image_array = image_array.squeeze(0)

        # Move to CPU, detach, convert to NumPy, and transpose to HWC format
        image_array = image_array.detach().cpu().numpy().transpose(1, 2, 0)

    # Ensure the image is in uint8 format and in [0, 255] range
    if image_array.max() <= 1.0:
        image_array = (image_array * 255).astype(np.uint8)
    else:
        image_array = image_array.astype(np.uint8)

    # Save the image using OpenCV
    success = cv2.imwrite(filename, image_array)
    if success:
        print(f"Image saved as {filename}")
    else:
        print("Failed to save the image. Please check the image format and path.")


# Save the image
save_image(up_samples, "/content/inpainted_img.png")
save_image(source_image_256, "/content/original_image.png")


In [ ]:
!pip install scikit-image

In [ ]:
import sys, os
import math
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage

In [ ]:
from math import log10, sqrt
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if mse == 0:  # MSE is zero means no noise is present in the signal
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def calculate_ssim(original, compressed):
    # Convert images to grayscale if needed for SSIM calculation
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    compressed_gray = cv2.cvtColor(compressed, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(original_gray, compressed_gray, full=True)
    return score

def main():
    # Load images with correct paths and extensions
    original = cv2.imread("/content/original_image.png")
    compressed = cv2.imread("/content/inpainted_img.png")

    # Check if images were loaded successfully
    if original is None or compressed is None:
        print("Error: One or both image files could not be loaded. Please check the file paths.")
        return

    # Resize original to match the compressed image's dimensions
    original_resized = cv2.resize(original, (compressed.shape[1], compressed.shape[0]))

    # Calculate PSNR
    psnr_value = PSNR(original_resized, compressed)
    print(f"PSNR value is {psnr_value} dB")

    # Calculate SSIM
    ssim_value = calculate_ssim(original_resized, compressed)
    print(f"SSIM value is {ssim_value}")

if __name__ == "__main__":
    main()
